In [2]:
!python export.py -o loop

Exported: /home/ck/code/loop/dev/00a_annotations.ipynb -> loop/annotations.py
Exported: /home/ck/code/loop/dev/00b_config.ipynb -> loop/config.py
Exported: /home/ck/code/loop/dev/00c_utils.ipynb -> loop/utils.py
Exported: /home/ck/code/loop/dev/00d_mixins.ipynb -> loop/mixins.py
Exported: /home/ck/code/loop/dev/01a_callbacks.ipynb -> loop/callbacks.py
Exported: /home/ck/code/loop/dev/01b_modules.ipynb -> loop/modules.py
Exported: /home/ck/code/loop/dev/02a_metrics.ipynb -> loop/metrics.py
Exported: /home/ck/code/loop/dev/02b_phase.ipynb -> loop/phase.py
Exported: /home/ck/code/loop/dev/02c_training.ipynb -> loop/training.py
Exported: /home/ck/code/loop/dev/03a_schedule.ipynb -> loop/schedule.py
Exported: /home/ck/code/loop/dev/03b_early_stopping.ipynb -> loop/early_stopping.py
Exported: /home/ck/code/loop/dev/03c_predictions.ipynb -> loop/predictions.py
Exported: /home/ck/code/loop/dev/99_testing.ipynb -> loop/testing.py
13 notebook(s) exported into folder: loop


In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [7]:
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset

from loop.config import defaults
from loop.training import Loop
from loop.utils import from_torch

In [ ]:
class Predictor:
    """Wraps model with """
    
    def __init__(self, model, device, batch_size: int=defaults.bs, loss_fn: 'callable'=None):
        model.to(device)
        self.model = model
        self.device = device
        self.batch_size = batch_size
        self.loss_fn = loss_fn
    
    @staticmethod
    def from_loop(self, loop: Loop, on_device=None):
        return Predictor(loop.model, on_device or loop.device, loop.loss_fn)
        
    def apply(self, ds: Dataset):
        dl = DataLoader(ds, shuffle=False, batch_size=self.batch_size)
        result, actual = [], []
        for xb, yb in dl:
            y_hat = self.model(xb.to(self.device))
            result += from_torch(y_hat).tolist()
            actual += from_torch(y_val).tolist()
        return np.array(result), np.array(actual)
    
    def predict(self, X):
        y = [0] * len(X)
        preds, _ = self.score(TensorDataset(X, y))
        return preds
    
    def score(self, X, y):
        return self.apply(TensorDataset(X, y))
    
    def loss(self, X, y, loss_fn=None):
        loss_fn = loss_fn or self.loss_fn
        if loss_fn is None:
            raise ValueError('cannot compute loss if self.loss_fn is None')
        preds, gt = self.score(X, y)
        return self.loss_fn(preds, gt)

In [8]:
from pathlib import Path

from loop.callbacks import Average
from loop.early_stopping import EarlyStopping, ModelSaver
from loop.metrics import accuracy
from loop.testing import get_mnist